In [2]:
from src.utils.submission import algonauts2021_submission_from_whole_brain_prediction

In [3]:
import torch
import numpy as np

import pandas as pd
from tqdm.auto import tqdm
from pathlib import Path

# read outputs

In [3]:
# read from notebook 1. and 2. (cross-notebook-ref)
tmp_dir = '/data/huze/ray_results/algonauts2021/ensemble_outputs'
tmp_dir = Path(tmp_dir)

In [4]:
prediction_dicts = []
for path in tmp_dir.iterdir():
    try:
        name = path.name.replace('-prediction.pt', '')
        d = {k: v for k, v in map(lambda kv: kv.split('='), name.split(','))}
        d['path'] = path
        prediction_dicts.append(d)
    except:
        continue

prediction_df = pd.DataFrame(prediction_dicts)

prediction_df.fillna('', inplace=True) # replace nan

In [5]:
prediction_df.head()

,he_sch,model_sch,roi_sch,path,cross_roi_sch
0,H3,multi_layer&i3d_rgb,WB,/data/huze/ray_results/algonauts2021/ensemble_...,
1,H1,single_layer&i3d_rgb+i3d_flow,ROIxSMC,/data/huze/ray_results/algonauts2021/ensemble_...,croi_ensemble
2,H3,single_layer&i3d_rgb,ROIxSMC,/data/huze/ray_results/algonauts2021/ensemble_...,voxel_swap
3,H3,single_layer+multi_layer&i3d_rgb,ROIxMC,/data/huze/ray_results/algonauts2021/ensemble_...,croi_ensemble
4,H3,multi_layer&i3d_flow,LC,/data/huze/ray_results/algonauts2021/ensemble_...,


In [6]:
prediction_df.nunique()

he_sch             3
model_sch          9
roi_sch           10
path             196
cross_roi_sch      4
dtype: int64

In [7]:
from pprint import pprint

for ckey in prediction_df.keys():
    if ckey == 'path':
        continue
    pprint(ckey)
    lst = prediction_df[ckey].dropna().unique().tolist()
    pprint(len(lst))
    pprint(lst)

'he_sch'
3
['H3', 'H1', 'H2']
'model_sch'
9
['multi_layer&i3d_rgb',
 'single_layer&i3d_rgb+i3d_flow',
 'single_layer&i3d_rgb',
 'single_layer+multi_layer&i3d_rgb',
 'multi_layer&i3d_flow',
 'single_layer+multi_layer&i3d_rgb+i3d_flow',
 'multi_layer&i3d_rgb+i3d_flow',
 'single_layer+multi_layer&i3d_flow',
 'single_layer&i3d_flow']
'roi_sch'
10
['WB', 'ROIxSMC', 'ROIxMC', 'LC', 'ROIxSC', 'ROIxLC', 'ROI', 'SMC', 'SC', 'MC']
'cross_roi_sch'
4
['', 'croi_ensemble', 'voxel_swap', 'croi_swap']


# submission

In [8]:
# only submit interesting configs

In [9]:
submission_configs_keys = ['model_sch', 'he_sch', 'roi_sch', 'cross_roi_sch']
submission_configs = []

table_configs = []
# is `ROI` working. 
# saved at notebook 1. (cross-notebook-ref)
table_configs.append([
    ('single_layer&i3d_rgb+i3d_flow', 'H3', roi_sch, 'croi_ensemble' if 'x' in roi_sch else '')
    for roi_sch in prediction_df['roi_sch'].dropna().unique().tolist()])
# is `cross ROI combine` working. 
# saved at notebook 2. (cross-notebook-ref)
table_configs.append([
    ('single_layer&i3d_rgb+i3d_flow', 'H3', 'ROIxSMC', 'croi_ensemble'),
    ('single_layer&i3d_rgb+i3d_flow', 'H3', 'ROIxSMC', 'croi_swap'),
    ('single_layer&i3d_rgb+i3d_flow', 'H3', 'ROIxSMC', 'voxel_swap'),
    ('single_layer&i3d_rgb+i3d_flow', 'H3', 'ROI', ''),
    ('single_layer&i3d_rgb+i3d_flow', 'H3', 'SMC', ''),])
# which variant of our ROI is best. 
# saved at notebook 1. and 2. (cross-notebook-ref)
table_configs.append([
    ('single_layer&i3d_rgb+i3d_flow', 'H3', roi_sch, 'croi_ensemble')
    for roi_sch in prediction_df['roi_sch'].dropna().unique().tolist() if 'x' in roi_sch])
# is `hierachy` working. 
# saved at notebook 1. and 2. (cross-notebook-ref)
table_configs.append([
    ('single_layer&i3d_rgb+i3d_flow', he_sch, 'ROIxSMC', 'croi_ensemble')
    for he_sch in prediction_df['he_sch'].dropna().unique().tolist()])
# model schematics. 
# saved at notebook 1. and 2. (cross-notebook-ref)
table_configs.append([
    (model_sch, 'H3', 'ROIxSMC', 'croi_ensemble') 
    for model_sch in prediction_df['model_sch'].dropna().unique().tolist()])

submission_configs = set(sum(table_configs, []))

In [17]:
torch.save(table_configs, './tmp/table_configs.pt')

In [10]:
submission_configs

{('multi_layer&i3d_flow', 'H3', 'ROIxSMC', 'croi_ensemble'),
 ('multi_layer&i3d_rgb', 'H3', 'ROIxSMC', 'croi_ensemble'),
 ('multi_layer&i3d_rgb+i3d_flow', 'H3', 'ROIxSMC', 'croi_ensemble'),
 ('single_layer&i3d_flow', 'H3', 'ROIxSMC', 'croi_ensemble'),
 ('single_layer&i3d_rgb', 'H3', 'ROIxSMC', 'croi_ensemble'),
 ('single_layer&i3d_rgb+i3d_flow', 'H1', 'ROIxSMC', 'croi_ensemble'),
 ('single_layer&i3d_rgb+i3d_flow', 'H2', 'ROIxSMC', 'croi_ensemble'),
 ('single_layer&i3d_rgb+i3d_flow', 'H3', 'LC', ''),
 ('single_layer&i3d_rgb+i3d_flow', 'H3', 'MC', ''),
 ('single_layer&i3d_rgb+i3d_flow', 'H3', 'ROI', ''),
 ('single_layer&i3d_rgb+i3d_flow', 'H3', 'ROIxLC', 'croi_ensemble'),
 ('single_layer&i3d_rgb+i3d_flow', 'H3', 'ROIxMC', 'croi_ensemble'),
 ('single_layer&i3d_rgb+i3d_flow', 'H3', 'ROIxSC', 'croi_ensemble'),
 ('single_layer&i3d_rgb+i3d_flow', 'H3', 'ROIxSMC', 'croi_ensemble'),
 ('single_layer&i3d_rgb+i3d_flow', 'H3', 'ROIxSMC', 'croi_swap'),
 ('single_layer&i3d_rgb+i3d_flow', 'H3', 'ROIxS

In [11]:
len(submission_configs)

22

In [12]:
from src.utils.misc import my_query_df

In [18]:
!rm -r submissions

In [14]:
submission_dicts = []
for i_submission, submission_config in tqdm(enumerate(submission_configs)):
    submission_config_dict = {k: v for k, v in zip(submission_configs_keys, submission_config)}
    # pprint(submission_config_dict)
    df = my_query_df(prediction_df, equal_dict=submission_config_dict)
    if len(df) == 0:
        print('warning: config with no output, please run notebook 1. and 2. first. config:', submission_config_dict)
        continue
    assert len(df) == 1
    path = df.path.item()
    name = path.name.replace('-prediction.pt', '')

    i_submission = f'{i_submission:03d}'
    submission_name = f'i={i_submission},{name}'
    submission_config_dict['i_submission'] = i_submission
    submission_config_dict['path'] = path
    submission_dicts.append(submission_config_dict)
    
    prediction = torch.load(path)
    prediction = prediction[1000:].float().numpy()

    algonauts2021_submission_from_whole_brain_prediction('../src/config/dataset',
                                                        submission_name,
                                                        prediction,
                                                        output_dir='./submissions/')

submission_df = pd.DataFrame(submission_dicts)

0it [00:00, ?it/s]

In [15]:
submission_df

,model_sch,he_sch,roi_sch,cross_roi_sch,i_submission,path
0,multi_layer&i3d_rgb,H3,ROIxSMC,croi_ensemble,000,/data/huze/ray_results/algonauts2021/ensemble_...
1,single_layer&i3d_rgb+i3d_flow,H3,ROIxSC,croi_ensemble,001,/data/huze/ray_results/algonauts2021/ensemble_...
2,single_layer&i3d_rgb+i3d_flow,H3,ROI,,002,/data/huze/ray_results/algonauts2021/ensemble_...
3,single_layer&i3d_rgb+i3d_flow,H3,WB,,003,/data/huze/ray_results/algonauts2021/ensemble_...
4,single_layer&i3d_flow,H3,ROIxSMC,croi_ensemble,004,/data/huze/ray_results/algonauts2021/ensemble_...
5,single_layer&i3d_rgb+i3d_flow,H1,ROIxSMC,croi_ensemble,005,/data/huze/ray_results/algonauts2021/ensemble_...
6,single_layer&i3d_rgb+i3d_flow,H3,ROIxLC,croi_ensemble,006,/data/huze/ray_results/algonauts2021/ensemble_...
7,multi_layer&i3d_flow,H3,ROIxSMC,croi_ensemble,007,/data/huze/ray_results/algonauts2021/ensemble_...
8,single_layer&i3d_rgb+i3d_flow,H3,SMC,,008,/data/huze/ray_results/algonauts2021/ensemble_...
9,single_layer&i3d_rgb+i3d_flow,H2,ROIxSMC,croi_ensemble,009,/data/huze/ray_results/algonauts2021/ensemble_...


In [16]:
torch.save(submission_df, './tmp/submission_df.pt')

# baseline

In [6]:
# from notebook 201. (corss-notebook-ref)
predictions_dict = {
    # from previous project, notebook 'plot mc fi score, maybe final'
    'Baseline,VGG16 2D + LSTM,NoEnsemble,layer=(x1,x2,x3,x4,x5),pathway=cascade,roi=WB': Path(
        '/data_smr/huze/projects/my_algonauts/predictions/bc4bd42bc70d409daebe27f2c0fae255/WB.pt'),
    # from notebook 1.
    'Baseline,Inflated 3D Resnet,RGB stream,NoEnsemble,layer=(x1,x2,x3,x4),pathway=topdown,roi=WB': Path(
        "/data/huze/ray_results/algonauts2021/algonauts2021_i3d_rgb-multi_layer/run_single_tune_config_575a8_00066_66_DATASET.ROI=WB,MODEL.BACKBONE.LAYERS=_'x1', 'x2', 'x3', 'x4'_,MODEL.BACKBONE.LAYER_PATHWAYS=_2022-02-26_14-18-59/prediction.npy"),
}
predictions_dict = {k: path for k, path in predictions_dict.items() if path.exists()}
assert len(predictions_dict) > 1

In [13]:
for name, path in predictions_dict.items():
    if path.name.endswith('.pt'):
        prediction = torch.load(path).float()
    else:
        prediction = torch.tensor(np.load(path)).float()
    
    submission_name = name
    prediction = prediction[1000:].numpy()

    algonauts2021_submission_from_whole_brain_prediction('../src/config/dataset',
                                                    submission_name,
                                                    prediction,
                                                    output_dir='./submissions/')

In [55]:
# draft

In [29]:
filter_dict = {
    'he_sch': ['H3'],
    'model_sch': ['single_layer&i3d_rgb+i3d_flow'],
    # 'roi_sch': ['ROI', 'LC', 'MC', 'SC', 'SMC'],
    'cross_roi_sch': ['croi_ensemble'],
}

In [30]:
filtered_df = df
for k, vs in filter_dict.items():
    print(k, vs)
    filtered_df = filtered_df.loc[filtered_df[k].isin(vs)]
    print(len(filtered_df))

he_sch ['H3']
68
model_sch ['single_layer&i3d_rgb+i3d_flow']
17
cross_roi_sch ['croi_ensemble']
4


In [31]:
filtered_df = filtered_df.sort_values(filtered_df.keys().tolist())

In [32]:
filtered_df

,he_sch,model_sch,roi_sch,cross_roi_sch,path
72,H3,single_layer&i3d_rgb+i3d_flow,ROIxLC,croi_ensemble,/data/huze/ray_results/algonauts2021/ensemble_...
1,H3,single_layer&i3d_rgb+i3d_flow,ROIxMC,croi_ensemble,/data/huze/ray_results/algonauts2021/ensemble_...
24,H3,single_layer&i3d_rgb+i3d_flow,ROIxSC,croi_ensemble,/data/huze/ray_results/algonauts2021/ensemble_...
36,H3,single_layer&i3d_rgb+i3d_flow,ROIxSMC,croi_ensemble,/data/huze/ray_results/algonauts2021/ensemble_...


In [33]:
filtered_df.iloc[-1].path

PosixPath('/data/huze/ray_results/algonauts2021/ensemble_outputs/he_sch=H3,model_sch=single_layer&i3d_rgb+i3d_flow,roi_sch=ROIxSMC,cross_roi_sch=croi_ensemble-prediction.pt')

In [25]:
!rm -r submissions

rm: cannot remove 'submissions': No such file or directory


In [26]:
for i_row, row in filtered_df.iterrows():
    path = row.path
    name = path.name.replace('-prediction.pt', '')
    
    prediction = torch.load(path)
    prediction = prediction[1000:].float().numpy()

    algonauts2021_submission_from_whole_brain_prediction('../src/config/dataset',
                                                        name,
                                                        prediction,
                                                        output_dir='./submissions/')